# GPT-2 для генерации новых Покемонов

Этот блокнот позволит вам преобразовать папку с изображениями в текстовый формат, а затем обучить модель GPT-2 на изображениях Покемонов.

Чтобы использовать этот блокнот, просто следуйте инструкциям и запускайте каждую ячейку в соответствии с указаниями.


## Установка зависимостей

Сначала импортируем все необходимые модули и скачиваем для переподготовки модель 124М GPT-2:

In [ ]:
!pip install -q gpt-2-simple

import sys
import math
from PIL import Image
from os import listdir
import shutil

%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files, drive

gpt2.download_gpt2(model_name="124M")

## Работа с Google Drive

Подключаем Google Drive (на нем уже есть датасет всех Покемонов с [kaggle](https://www.kaggle.com/kvpratama/pokemon-images-dataset))

In [ ]:
drive.mount('/content/drive')

## Сохраняем картинки в текст для генерации (на данный момент самый быстрый вариант)

Теперь мы конвертируем наши изображения Покемонов в один длинный текстовый файл, который мы будем использовать для обучения. По умолчанию мы используем папку на Google Диске под названием «pokemons».

Измените это на имя папки на вашем Google Диске, которую вы хотите преобразовать, затем запустите ячейку:

In [ ]:
output = ''
input_folder = 'pokemons'
input_folder_path = f'/content/drive/My Drive/{input_folder}'

for f in listdir(input_folder_path):
    ext = f.split('.')[-1]

    print(f'Превращаем файл {f}')

    path = f"{input_folder_path}/{f}"

    image = Image.open(path).convert('RGBA')
    pixels = image.load()
    width, height = image.size[0], image.size[1]

    lines = []

    for y in range(height):
        padLength = len(str(width))
        padded = str(y).zfill(padLength)
        split = [padded + 'd']

        for x in range(width):
            color = pixels[x, y]

            s = '~'

            if len(color) < 4 or color[3] > 128:
                r = color[0]
                g = color[1]
                b = color[2]

                if r == 85 and g == 85 and b == 85:
                    s = 'a'

                elif r == 170 and g == 170 and b == 170:
                    s = 'b'

                else:
                    mR = math.floor(r / 64)
                    mG = math.floor(g / 64)
                    mB = math.floor(b / 64)

                    c = 0;

                    c += mR
                    c = c << 2
                    c += mG
                    c = c << 2
                    c += mB

                    s = chr(c+33)

            split.append(s)

        lines.append(' '.join(split))

    reversed = []
    for line in lines:
        reversed.insert(0, (line.replace('d ', 'u ', 1)))

    output += '\n'.join(lines)
    output += '\n'.join(reversed)

print('Сохранение...')

with open('images.txt', 'w') as output_file:
    output_file.write(output)

print('Выполненно!')

## Обучаем модель

Теперь обучим модель GPT-2 на преобразованных изображениях.

Создадим папку `pokemon-gpt-2-checkpoints` в которую мы будем сохранять на Google Диске.

Количество шагов определяет, как долго мы тренируем нашу модель на этих данных. Чем дольше, тем лучше результат.


In [ ]:
checkpoint_folder = 'pokemon-gpt-2-checkpoints'
run_name = 'pokemon-gpt-2-run'
steps = 2000

session = gpt2.start_tf_sess()

session = gpt2.reset_session(session)

gpt2.finetune(session,
    dataset='all-images.txt',
    model_name='124M',
    steps=steps,
    restore_from='latest',
    run_name=run_name,
    print_every=10,
    sample_every=500,
    save_every=200
  )

shutil.copytree(f'checkpoint/{run_name}', f'/content/drive/My Drive/{checkpoint_folder}/{run_name}')